# Integrating document loaders

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import HNLoader

## PDF document loaders

In [ ]:
# Create a document loader for attention_is_all_you_need.pdf
loader = PyPDFLoader("documents/attention-is-all-you-need.pdf")

# Load the document
data = loader.load()
print(data[0])

## CSV document loaders

In [ ]:
# Create a document loader for fifa_countries_audience.csv
loader = CSVLoader(file_path='documents/fifa_countries_audience.csv')

# Load the document
data = loader.load()
print(data[0])

## Third-party document loaders

In [ ]:
# Create a document loader for the top Hacker News stories
loader = HNLoader("https://news.ycombinator.com")

# Load the document
data = loader.load()

# Print the first document
print(data[0])

# Print the first document's metadata
print(data[0].metadata)